In [ ]:
import os, time, re
from pathlib import Path

from langchain.document_loaders import UnstructuredMarkdownLoader
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter, MarkdownHeaderTextSplitter
from langchain.vectorstores import Chroma
from langchain.schema import Document
from langchain_huggingface import HuggingFaceEmbeddings
from transformers import AutoTokenizer

# ====== 基本設定 ======
markdown_folder = Path("TSpec-LLM/Rel-16")
model_name = "sentence-transformers/all-MiniLM-L6-v2"
embedding = HuggingFaceEmbeddings(model_name=model_name, model_kwargs={"device": "cuda"})

# 用 HF tokenizer 做「token 為單位」的長度估算
tokenizer = AutoTokenizer.from_pretrained(model_name)
def token_len(text: str) -> int:
    return len(tokenizer.encode(text, add_special_tokens=False))

# 章節標題切割（先粗切）
md_header_splitter = MarkdownHeaderTextSplitter(
    headers_to_split_on=[
        ("#", "h1"),
        ("##", "h2"),
        ("###", "h3"),
        ("####", "h4"),
    ]
)

# 二階段遞迴切割（再細切）：目標 800 tokens、重疊 120 tokens
rec_splitter = RecursiveCharacterTextSplitter(
    # 優先用 Markdown 結構斷點，最後才退回句子/固定長度
    separators=[
        "\n#### ", "\n### ", "\n## ", "\n# ",                 # Markdown 標題
        "\n\n",                                               # 段落
        "\n- ", "\n* ",                                       # 列表
        "\n",                                                 # 單行
        ". ", "? ", "! ",                                     # 句子
        " ",                                                  # 空白
        ""                                                    # 保底
    ],
    chunk_size=800,
    chunk_overlap=120,
    length_function=token_len,
    is_separator_regex=False,
)

# 你想測的 chunk 尺寸組（會產出多個 DB 版本）
chunk_sizes = [300, 500, 700, 900, 1100]
# 只要改這個就能固定 overlap 與 token 化計算邏輯
def make_rec_splitter(sz: int):
    return RecursiveCharacterTextSplitter(
        separators=[
            "\n#### ", "\n### ", "\n## ", "\n# ",
            "\n\n", "\n- ", "\n* ", "\n",
            ". ", "? ", "! ", " ", ""
        ],
        chunk_size=sz,
        chunk_overlap=min(150, max(50, int(sz*0.15))),  # overlap 取 10–15% 上下界 50~150
        length_function=token_len,
        is_separator_regex=False,
    )

# ====== 掃描 Markdown 檔 ======
markdown_files = [p for p in markdown_folder.rglob("*.md")]
print(f"📁 共發現 {len(markdown_files)} 個 Markdown 檔案")

start_time = time.time()
all_chunk_summary = {}

# ====== 主流程：依不同 chunk_size 產生一個「單一」Chroma DB ======
for chunk_size in chunk_sizes:
    print(f"\n🚀 開始處理 chunk_size = {chunk_size}")
    persist_dir = Path(f"3gpp_db_tokens_{chunk_size}")
    persist_dir.mkdir(parents=True, exist_ok=True)

    collection_name = f"r16_specs_tok{chunk_size}"
    # 單一資料庫、單一 collection（避免「每個 spec 一個 DB」的小庫分散問題）
    vectordb = None
    doc_count = 0
    chunk_summary = {}

    rec_splitter_sz = make_rec_splitter(chunk_size)

    for i, full_path in enumerate(markdown_files, 1):
        spec_path = Path(full_path)
        spec_name = spec_path.stem  # 例如 38463-gf0
        print(f"🔹 處理第 {i}/{len(markdown_files)} 個：{spec_name}")

        # 載入
        loader = UnstructuredMarkdownLoader(str(spec_path))
        raw_docs = loader.load()
        if not raw_docs:
            print(f"⚠️ {spec_name} 沒內容，跳過…")
            continue

        # 補充基礎 metadata
        for d in raw_docs:
            d.metadata.update({
                "spec": spec_name,
                "path": str(spec_path),
                "release": "Rel-16",
                # 可從檔名粗抽 38.xxx
                "series": re.findall(r"^(\d{5})", spec_name)[0] if re.findall(r"^(\d{5})", spec_name) else ""
            })

        # 先用 MarkdownHeaderTextSplitter 依章節粗切
        chapter_docs: list[Document] = []
        for d in raw_docs:
            sections = md_header_splitter.split_text(d.page_content)
            if not sections:  # 萬一沒有標題就當成整篇
                chapter_docs.append(d)
            else:
                for s in sections:
                    # 把章節標題集合成可檢索的欄位
                    title_chain = " / ".join([f"{k}:{v}" for k, v in s.metadata.items() if k.startswith("h")])
                    chapter_docs.append(
                        Document(
                            page_content=s.page_content.strip(),
                            metadata={
                                **d.metadata,
                                "section_title": title_chain,
                            }
                        )
                    )

        # 再用遞迴切割到目標 token 長度
        splits: list[Document] = []
        for ch in chapter_docs:
            # 在內容前面加上「規範 + 章節題名」，可顯著提高檢索分數
            prefix = ""
            if ch.metadata.get("spec") or ch.metadata.get("section_title"):
                prefix = f"[{ch.metadata.get('spec','')}] {ch.metadata.get('section_title','')}\n"
            chunks = rec_splitter_sz.split_text(prefix + ch.page_content)

            for idx, chunk in enumerate(chunks):
                splits.append(
                    Document(
                        page_content=chunk.strip(),
                        metadata={
                            **ch.metadata,
                            "chunk_index": idx,
                            "chunk_size_tokens": chunk_size
                        }
                    )
                )

        chunk_summary[spec_name] = len(splits)
        doc_count += len(splits)
        print(f"✅ 切割完成：{spec_name}，共 {len(splits)} 個 chunks")

        if not splits:
            continue

        # 寫入同一個 Chroma collection
        if vectordb is None:
            vectordb = Chroma.from_documents(
                documents=splits,
                embedding=embedding,
                collection_name=collection_name,
                persist_directory=str(persist_dir),
            )
        else:
            vectordb.add_documents(splits)

    if vectordb is not None:
        vectordb.persist()
        print(f"💾 已儲存至 {persist_dir} / collection={collection_name}，總文件 {doc_count}")

    all_chunk_summary[chunk_size] = chunk_summary

end_time = time.time()

# ====== 統計輸出 ======
print("\n📊 Chunk 統計總結：")
for chunk_size, summary in all_chunk_summary.items():
    print(f"\n🔸 chunk_size = {chunk_size}")
    total_chunks = sum(summary.values())
    for spec, count in sorted(summary.items()):
        print(f" - {spec.ljust(22)}: {count} chunks")
    print(f"✅ 共產生 {total_chunks} chunks")
print(f"\n⏱️ 總處理時間：{end_time - start_time:.2f} 秒")


2025-08-12 02:01:41.318028: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-08-12 02:01:41.336589: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-08-12 02:01:41.336613: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-08-12 02:01:41.337233: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-08-12 02:01:41.340460: I tensorflow/core/platform/cpu_feature_guar

📁 共發現 1697 個 Markdown 檔案

🚀 開始處理 chunk_size = 300
🔹 處理第 1/1697 個：46081-g00
✅ 切割完成：46081-g00，共 18 個 chunks


Token indices sequence length is longer than the specified maximum sequence length for this model (1725 > 512). Running this sequence through the model will result in indexing errors


🔹 處理第 2/1697 個：46053-g00
✅ 切割完成：46053-g00，共 12 個 chunks
🔹 處理第 3/1697 個：46051-g00
✅ 切割完成：46051-g00，共 22 個 chunks
🔹 處理第 4/1697 個：46062-g00
✅ 切割完成：46062-g00，共 37 個 chunks
🔹 處理第 5/1697 個：46020-g00
✅ 切割完成：46020-g00，共 154 個 chunks
🔹 處理第 6/1697 個：46011-g00
✅ 切割完成：46011-g00，共 8 個 chunks
🔹 處理第 7/1697 個：46008-g00
✅ 切割完成：46008-g00，共 58 個 chunks
🔹 處理第 8/1697 個：46002-g00
✅ 切割完成：46002-g00，共 16 個 chunks
🔹 處理第 9/1697 個：46042-g00
✅ 切割完成：46042-g00，共 45 個 chunks
🔹 處理第 10/1697 個：46061-g00
✅ 切割完成：46061-g00，共 17 個 chunks
🔹 處理第 11/1697 個：46010-g00
✅ 切割完成：46010-g00，共 178 個 chunks
🔹 處理第 12/1697 個：46076-g00
✅ 切割完成：46076-g00，共 156 個 chunks
🔹 處理第 13/1697 個：46082-g00
✅ 切割完成：46082-g00，共 37 個 chunks
🔹 處理第 14/1697 個：46022-g00
✅ 切割完成：46022-g00，共 26 個 chunks
🔹 處理第 15/1697 個：46041-g00
✅ 切割完成：46041-g00，共 21 個 chunks
🔹 處理第 16/1697 個：46032-g00
✅ 切割完成：46032-g00，共 100 個 chunks
🔹 處理第 17/1697 個：46085-g00
✅ 切割完成：46085-g00，共 97 個 chunks
🔹 處理第 18/1697 個：readme
✅ 切割完成：readme，共 1 個 chunks
🔹 處理第 19/1697 個：46007-g01
✅ 切割完成：46007-g01，

/tmp/ipykernel_2860044/1896218737.py:169: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vectordb.persist()


✅ 切割完成：46081-g00，共 11 個 chunks
🔹 處理第 2/1697 個：46053-g00
✅ 切割完成：46053-g00，共 8 個 chunks
🔹 處理第 3/1697 個：46051-g00
✅ 切割完成：46051-g00，共 14 個 chunks
🔹 處理第 4/1697 個：46062-g00
✅ 切割完成：46062-g00，共 23 個 chunks
🔹 處理第 5/1697 個：46020-g00
✅ 切割完成：46020-g00，共 94 個 chunks
🔹 處理第 6/1697 個：46011-g00
✅ 切割完成：46011-g00，共 4 個 chunks
🔹 處理第 7/1697 個：46008-g00
✅ 切割完成：46008-g00，共 32 個 chunks
🔹 處理第 8/1697 個：46002-g00
✅ 切割完成：46002-g00，共 10 個 chunks
🔹 處理第 9/1697 個：46042-g00
✅ 切割完成：46042-g00，共 26 個 chunks
🔹 處理第 10/1697 個：46061-g00
✅ 切割完成：46061-g00，共 10 個 chunks
🔹 處理第 11/1697 個：46010-g00
✅ 切割完成：46010-g00，共 104 個 chunks
🔹 處理第 12/1697 個：46076-g00
✅ 切割完成：46076-g00，共 96 個 chunks
🔹 處理第 13/1697 個：46082-g00
✅ 切割完成：46082-g00，共 22 個 chunks
🔹 處理第 14/1697 個：46022-g00
✅ 切割完成：46022-g00，共 14 個 chunks
🔹 處理第 15/1697 個：46041-g00
✅ 切割完成：46041-g00，共 12 個 chunks
🔹 處理第 16/1697 個：46032-g00
✅ 切割完成：46032-g00，共 58 個 chunks
🔹 處理第 17/1697 個：46085-g00
✅ 切割完成：46085-g00，共 59 個 chunks
🔹 處理第 18/1697 個：readme
✅ 切割完成：readme，共 1 個 chunks
🔹 處理第 19/1697 個：